In [13]:
pip install gradio

In [12]:
pip install pymorphy2

In [37]:
import pymorphy2
import gradio as gr
import re

morph = pymorphy2.MorphAnalyzer()

pos_translation = {
    'NOUN': 'іменник',
    'VERB': 'дієслово',
    'ADJF': 'прикметник',
    'INFN': 'інфінітив',
    'NUMR': 'числівник',
    'ADVB': 'прислівник',
    'NPRO': 'займенник',
    'PREP': 'прийменник',
    'CONJ': 'сполучник',
    'PRCL': 'частка',
    'INTJ': 'вигук',
}

gender_translation = {
    'masc': 'чоловічий',
    'femn': 'жіночий',
    'neut': 'середній',
}

number_translation = {
    'sing': 'однина',
    'plur': 'множина',
}

case_translation = {
    'nomn': 'називний',
    'gent': 'родовий',
    'datv': 'давальний',
    'accs': 'знахідний',
    'ablt': 'орудний',
    'loct': 'місцевий',
    'voct': 'кличний',
    'gen2': 'родовий (альтернативний)',
    'acc2': 'знахідний (альтернативний)',
    'loc2': 'місцевий (альтернативний)',
}

tense_translation = {
    'past': 'минулий',
    'pres': 'теперішній',
    'futr': 'майбутній',
}

voice_translation = {
    'actv': 'активний',
    'pssv': 'пасивний',
}

aspect_translation = {
    'perf': 'доконаний',
    'impf': 'недоконаний',
}

def analyze_word(word):
    parsed_word = morph.parse(word)[0]
    pos = parsed_word.tag.POS
    info = {
        'Слово': word
    }
    if word in ['і', 'що']:
        info['Частина мови'] = 'сполучник'
    elif pos == 'VERB' and (word.endswith('ти') or word.endswith('є') or word.endswith('ть') or word.endswith('ться')):
        info['Частина мови'] = 'дієслово'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Число'] = number_translation.get(parsed_word.tag.number, 'невизначене')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
        info['Час'] = tense_translation.get(parsed_word.tag.tense, 'невизначений')
        info['Інфінітив'] = 'так' if 'INFN' in parsed_word.tag else 'ні'
    elif pos == 'NOUN':
        info['Частина мови'] = 'іменник'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Число'] = number_translation.get(parsed_word.tag.number, 'невизначене')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
    elif pos == 'ADJF':
        info['Частина мови'] = 'прикметник'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
    elif pos == 'NUMR':
        info['Частина мови'] = 'числівник'
    elif pos == 'ADVB':
        info['Частина мови'] = 'прислівник'
    elif pos == 'NPRO':
        info['Частина мови'] = 'займенник'
    elif pos == 'PREP':
        info['Частина мови'] = 'прийменник'
    elif pos == 'PRCL':
        info['Частина мови'] = 'частка'
    elif pos == 'CONJ':
        info['Частина мови'] = 'сполучник'
    elif pos == 'INTJ':
        info['Частина мови'] = 'вигук'

    return {key: value for key, value in info.items() if value is not None}

def process_text(text):
    words = re.findall(r'\b\w+\b', text)
    results = [analyze_word(word) for word in words]
    formatted_result = '\n\n'.join(
        [
            '{}:\n{}'.format(result['Слово'], '\n'.join(f'{k}: {v}' for k, v in result.items() if k != 'Слово'))
            for result in results
        ]
    )
    return formatted_result

with gr.Blocks() as demo:
    gr.Markdown('# Аналіз граматичної інформації')
    gr.Markdown('Цей модуль визначає граматичну інформацію слів українською мовою🤍.')
    with gr.Row():
        text_input = gr.Textbox(label='Введіть текст для аналізу🐼', placeholder='Напишіть слово або речення...', lines=4)
        text_output = gr.Textbox(label='Граматична інформація🐯', placeholder='Результат', lines=4)
    process_button = gr.Button('Обробити')
    process_button.click(fn=process_text, inputs=text_input, outputs=text_output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://453d64a7b37709f26b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
